In [129]:
from MovistarClient import MovistarClient
import httpx
import pandas as pd
from movi_payload_decode import decode_payload, encode_payload
import json
from datetime import datetime
import requests

In [130]:
mClient = MovistarClient()
aclient = httpx.AsyncClient()

In [155]:
                                        #CAMBIAR CEDULAS POR CADA PETICION
res = await mClient.get_clients(aclient, info_list= ['0504381088'], info_rule='contiene')

0504381088 id_number contiene


In [156]:
clients_df = pd.DataFrame(columns=['custumerId', 'name', 'id_number', 'phone_number', 'phone_status', 'contextId'])
for r in res:
    body = json.loads(r.json()['responses'][0]['body'])
    for key, value in body['data'].items():
        name = value['data']['-1']['v']['text']
        id_number = value['data']['9144718640113557588']['v']
        nombers = value['data']['9145139830513734387']
        custumerId = key[-20:-1]
        if nombers:
            for phone_object in nombers['v']:
                phone_number = phone_object['text']
                phone_status = phone_object['line_status_text']
                contextId = phone_object['line_id'][-13:len(phone_object['line_id'])]
                client_row = pd.DataFrame({'custumerId':custumerId, 'name':name, 'id_number':id_number, 'phone_number':phone_number, 'phone_status':phone_status, 'contextId':contextId}, index=[0])
                clients_df = pd.concat([clients_df, client_row], ignore_index=True)
        else:
            client_row = pd.DataFrame({'custumerId':custumerId, 'name':name, 'id_number':id_number, 'phone_number':'', 'phone_status':'', 'contextId':''}, index=[0])
            clients_df = pd.concat([clients_df, client_row], ignore_index=True)

In [158]:
custumerIds = clients_df['custumerId'].tolist()
contextIds = clients_df['contextId'].tolist()

In [159]:
rp = await mClient.get_lines_info(aclient, custumerIds, contextIds)

In [230]:
a = rp[0].json()
a

{'pageDataRequest': {'pageToken': '!board',
  'params': {'customerId': '9149095182513799619',
   'contextId': '1xkvf3qn6bv7r'}},
 'data': {'com.netcracker.csrdesktop.spa.shared.dto.CustomerBarDto': {'@class': 'com.netcracker.csrdesktop.spa.shared.dto.CustomerBarDto',
   'extData': {'@class': 'com.netcracker.solutions.tfnecu.csrd.spa.shared.dto.TfnecuCustomerBarExtDto',
    'relationshipFlag': False,
    'loyaltyProgramFlag': False,
    'loyaltyPoints': None,
    'primaryContact': 'EDWIN FABRICIO RECALDE HERRERA',
    'documentInfo': 'Cédula: 0504381088',
    'preferContactMethod': 'LUCHIN_RACALDE@HOTMAIL.COM',
    'contactHours': None,
    'contactTooltip': 'csrd-contact-method_email',
    'accountManagerName': None,
    'ejecutivo': None,
    'accountManagerUrl': None,
    'ejecutivoUrl': None,
    'customerSince': '14/04/2014',
    'customerCategory': 'Individual',
    'loyaltyTooltip': None,
    'canHaveLP': True,
    'hasAccessToRoe': True,
    'customerNameStyle': 'name_length_50'

In [ ]:
        # VARIABLES PARA PAYLOADS - UBICACION, FACTURACION, 
line_id = a['data']['com.netcracker.csrdesktop.spa.shared.dto.BoardContentDto']['csrdContext']['ctx_cihm_line_id']
customerId = a['data']['com.netcracker.csrdesktop.spa.shared.dto.BoardContentDto']['csrdContext']['ctx_csrd_page_params']['customerId']
contextId = a['data']['com.netcracker.csrdesktop.spa.shared.dto.BoardContentDto']['csrdContext']['ctx_csrd_page_params']['contextId']
ctx_cihm_customer_id = a['data']['com.netcracker.csrdesktop.spa.shared.dto.BoardContentDto']['csrdContext']['ctx_cihm_customer_id']
billing_account_id = a['data']['com.netcracker.csrdesktop.spa.shared.dto.BoardContentDto']['csrdContext']['ctx_cihm_billing_account_id']

In [228]:
line_id, customerId, contextId, ctx_cihm_customer_id, billing_account_id

('9158211702313211079',
 '9149095182513799619',
 '1xkvf3qn6bv7r',
 '9149095182513799619',
 '9149095236013542262')

In [238]:
all_dfs = []

for i, response in enumerate(rp):
    resp = response.json()
    try:
        data = resp['data']
    except KeyError:
        continue
    dt = data['com.netcracker.csrdesktop.spa.shared.dto.ContextsDto']['contextTabs']
    fc = data['com.netcracker.solutions.tfnecu.csrd.spa.shared.dto.SubscriberInformationDto']
    bc = data['com.netcracker.csrdesktop.spa.shared.dto.BoardContentDto']

    fecha_C_linea = fc.get('actualStartDate') # VERIFICAOD

    if len(dt) >= 2:
        dt_combined = {
            'id_tel': dt[1]['id'],  # MOSTRAR PARA REFERENCIA TABLA DE TELEFONOS
            'telefono': dt[1]['title'],
            'Estado_tel': dt[1]['statusText'],
            'id': dt[0]['id'], # MOSTRAR PARA REFERENCIA TABLA DE TELEFONOS
            'Cuenta de Linea': dt[0]['title'],
            'Estado Linea': dt[0]['statusText'],
            'Fecha inicio': fecha_C_linea,
            'billing_account_id': bc['csrdContext']['ctx_cihm_billing_account_id'],
            'line_id':bc['csrdContext']['ctx_cihm_line_id'],
        }
        df_c = pd.DataFrame([dt_combined])
    else:
        df_c = pd.DataFrame([{
            'id_tel': '',          # MOSTRAR PARA REFERENCIA TABLA DE TELEFONOS
            'telefono': '',
            'Estado_tel': '',
            'id': dt[0]['id'] if dt else '',       # MOSTRAR PARA REFERENCIA TABLA DE TELEFONOS
            'Cuenta de Linea': dt[0]['title'] if dt else '',
            'Estado Linea': dt[0]['statusText'] if dt else '',
            'Fecha inicio': fecha_C_linea,
        }])
    
    all_dfs.append(df_c)
    #df_filter_c = df_c[df_c['Cuenta_linea'].str.contains('Pospago|Prepago', na=False)]
    df = pd.concat(all_dfs, ignore_index=True)

In [241]:
comparar = pd.merge(clients_df, df, left_on='phone_number', right_on='telefono', how='left') # omito custumerId, contextId, telefono(REFERENCIA) y Estado_tel 
mostrar = comparar[[ 'name', 'id_number', 'phone_number', 'phone_status','Cuenta de Linea', 'Estado Linea', 'Fecha inicio']]

In [240]:
comparar = pd.read_csv('comparar.csv', dtype=str)

In [163]:
res = await mClient.get_custumers_info(aclient, custumerIds=comparar['custumerId'].drop_duplicates().tolist())
res

[<Response [200 OK]>]

In [164]:
#port out
port_out_df = pd.DataFrame()

status_dict = {
    "9145181582713012278": "Ingresando",
    "9145157196513977239": "Procesando",
    "9145157196513977240": "Pendiente en ASCP",
    "9145157196513977241": "Pendiente en ASCP como Aprobado",
    "9145157196513977242": "Pendiente en ASCP como Rechazado",
    "9145157196513977243": "Aprobado",
    "9145157196513977244": "Rechazado",
    "9145157196513977245": "Cancelado",
    "9157813640013022352": "Parcialmente rechazada"
}

def get_status_text(estado):
    return status_dict.get(estado, "Desconocido")

for r in res:
    body = json.loads(r.json()['responses'][1]['body'])
    data = body['data']
    if not data:
        continue
    for key, value in data.items():
        text = value['data']['-1']['v']['text']
        numero =value['data']['9145173418813998592']['v'][0]['text']
        estado = value['data']['9145157196513977236']['v']['v']
        df = pd.DataFrame({'name':text, 'phone_number':numero, 'estado':get_status_text(estado)}, index=[0])
        port_out_df = pd.concat([port_out_df, df], ignore_index=True)

In [165]:
#port in
port_in_df = pd.DataFrame()
for r in res:
    body = json.loads(r.json()['responses'][0]['body'])
    data = body['data']
    if not data:
        continue
    for key, value in data.items():
        text = value['data']['-1']['v']['text']
        numero =value['data']['9145173418813998592']['v'][0]['text']
        estado = value['data']['9145157196513977236']['v']['v']
        df = pd.DataFrame({'name':text, 'phone_number':numero, 'estado':estado}, index=[0])
        port_in_df = pd.concat([port_in_df, df], ignore_index=True)

In [166]:
# cambiar historial
hist_df = pd.DataFrame()
for r in res:
    data = json.loads(r.json()['responses'][2]['body'])['data']
    for key, value in data.items():
        try:
            date_ts = int(value['data']['9157884418713223833_9145589430013662246']['v'])/1000
            date = datetime.fromtimestamp(date_ts)
        except KeyError:
            date_ts = None
        try:
            nombre = value['data']['9072741107013063688']['v']['text']
        except KeyError:
            nombre = None
        try:
            number = value['data']['9144543752013303034']['v']['text']
        except KeyError:
            number = None
        try:
            objectTypeId = value['data']['9072741107013063688']['v']['objectTypeId']
        except KeyError:
            objectTypeId = None
        try:
            objectId = value['data']['9072741107013063688']['v']['objectId']
        except KeyError:
            objectId = None
        
        hist_df = pd.concat([hist_df, pd.DataFrame({'name':nombre, 'phone_number':number, 'date':date, 'objectTypeId':objectTypeId, 'objectId':objectId}, index=[0])], ignore_index=True)


In [167]:
df_entrada_orden = hist_df.sort_values(by='date', ascending=False).drop_duplicates(subset=['phone_number', 'name'], keep='first').reset_index(drop=True)

In [168]:
df_entrada_orden # incluir ncObjectID, custumerId

,name,phone_number,date,objectTypeId,objectId
0,PREPAGO TODO EN UNO [984343125],984343125,2024-11-01 14:40:32,9126083628313449001,9171028743613165174
1,PREVIOPAGO PLAN MOVISTAR ESSENTIAL [984343125],984343125,2024-09-26 17:42:30,9126083628313449001,9170829818713558057
2,Pago por cambio de plan Previopago #1035472125,984343125,2024-09-26 17:42:30,9126083628313449001,9171028743613165207
3,PREVIOPAGO BONO PERMANENCIA ESSENTIAL #1035472127,None,2024-09-26 17:42:30,9126083628313449001,9171028743613165204
4,PREPAGO TODO EN UNO [992608783],992608783,2024-08-08 05:30:22,9126083628313449001,9160176637713161223
5,PREPAGO TODO EN UNO [987930631],987930631,2024-08-08 04:01:22,9126083628313449001,9159037687013175449
6,PREPAGO TODO EN UNO [983271994],983271994,2024-06-29 15:04:28,9126083628313449001,9158211702313211079
7,Pago por Cambio de Plan #0976405680,984343125,2024-01-05 11:08:34,9126083628313449001,9168736809113396329
8,BONO 2GB PERMANENCIA #0976405682,None,2024-01-05 11:08:34,9126083628313449001,9168736809113396365
9,PLAN MOVISTAR STARTER PLUS. [984343125],984343125,2024-01-05 11:08:34,9126083628313449001,9165411563513864372


In [236]:
df = pd.merge(comparar, df_entrada_orden, on='phone_number', how='left')

In [56]:
res[4].json()['responses'][0]['body']

'{"metamodel":{"c":"com.netcracker.platform.ui.plugins.rest.service.transport.metamodel.MetaModel"},"c":"com.netcracker.platform.ui.plugins.rest.service.transport.response.TableCtrlResponse","data":{},"removed_ids":[],"filteringDescriptor":{"c":"com.netcracker.platform.ui.plugins.gwt.tablectrl.shared.transport.filtering.FilteringDescriptor","filters":{}},"root_ids":[],"dataErrors":{},"maxTableSize":1000,"totalRows":0,"message":"","objectPKs":[],"nextPageFirstRow":0,"responseParams":{"widgetContainerId":"9145244403913687320"},"sortingDescriptor":{"c":"com.netcracker.platform.ui.plugins.gwt.tablectrl.shared.transport.SortingDescriptor","sortings":{"4063077486013004590":{"c":"com.netcracker.platform.ui.plugins.gwt.tablectrl.shared.transport.SortingDescriptorItem"}}},"config":{"userViewData":"{\\"options\\":[{\\"isDefault\\":\\"false\\",\\"name\\":\\"default\\",\\"containerId\\":\\"9145244403913687320\\",\\"value\\":\\"\\"},{\\"isDefault\\":\\"true\\",\\"name\\":\\"Solicitudes de Port-In\\